In [ ]:
import numpy as np
import random as rd
import autograd.numpy as np
from matplotlib.pyplot import ion, draw, Rectangle, Line2D
import matplotlib.pyplot as plt
from CartPole import *
from scipy.optimize import minimize
import sobol_seq

# Task 2.3

In [ ]:
def model_pred_control(p):
    real_cart = CartPole()
    real_cart.setState([0, 0, 0.1, 0])
    real_state= np.zeros([15,5])
    l = 0
    for i in range(15):
        l += real_cart.loss()
        f = np.dot(p, real_cart.getState())
        real_state[i, :4] =real_cart.getState()
        real_state[i, -1] = f
        real_cart.performAction(f)
        real_cart.remap_angle()
    return l
l=model_pred_control([2,2,10,3])

In [ ]:
#Visualising minimums
n_s = 500
l_c = np.zeros([n_s,1])
p_c=np.zeros([n_s,4])
for i in range(n_s):
    pc = [1,np.random.uniform(-50,100),1,1]
    p_c[i] = pc
    l_c[i]=model_pred_control(pc)

plt.scatter(p_c[:,1],l_c)
plt.xlabel('p2')
plt.ylabel('Loss')
plt.xlim(-50,20)
plt.title('Varying p2, concerneed with cart velocity')
plt.show()

n_s = 500
l_c0 = np.zeros([n_s,1])
p_c0=np.zeros([n_s,4])
for i in range(n_s):
    pc0 = [np.random.uniform(-1000,1000),1,1,1]
    p_c0[i] = pc0
    l_c0[i]=model_pred_control(pc0)


plt.scatter(p_c0[:,0],l_c0)
plt.xlabel('p1')
plt.ylabel('Loss')
plt.xlim(-1000,1000)
plt.title('Varying p1, concerned with cart location')
plt.show()


### Contour PLots

In [ ]:
labels = ['p1','p2','p3','p4']
plt.tricontourf(p_cl[:,1],p_cl[:,2],l_cl[:,0],1000)
plt.xlabel(labels[1])
plt.ylabel(labels[2])
plt.ylim(-200,200)
#plt.xlim(0,80)
plt.colorbar()
plt.title('Loss function, Varying p2,p3')
plt.show()

### Optimizing with initial conditions

In [ ]:
p_1 = [1,8,10,10]
res = minimize(model_pred_control, p_1, method='nelder-mead',
               options={'maxiter':3000,'xatol': 1e-5, 'disp': True})

Optimization terminated successfully.
         Current function value: 11.811695
         Iterations: 138
         Function evaluations: 249


### Time Evolution

In [ ]:
def model_pred_control_2(p):
    real_cart = CartPole()
    real_cart.setState([0, 0, 0.1, 0])
    real_state= np.zeros([50,5])
    l = 0
    for i in range(50):
        l += real_cart.loss()
        f = np.dot(p, real_cart.getState())
        real_state[i, :4] =real_cart.getState()
        real_state[i, -1] = f
        real_cart.performAction(f)
        real_cart.remap_angle()
    return real_state

def model_pred_control_3(p,s):
    real_cart = CartPole()
    real_cart.setState(s)
    real_state= np.zeros([50,5])
    l = 0
    for i in range(50):
        l += real_cart.loss()
        f = np.dot(p, real_cart.getState())
        real_state[i, :4] =real_cart.getState()
        real_state[i, -1] = f
        real_cart.performAction(f)
        real_cart.remap_angle()
    return real_state

cc = model_pred_control_3([0.7051,1.21512,16.311,2.506],[0,0,-0.2,0])
aa=model_pred_control_2([0.7051,1.21512,16.311,2.506])
bb = model_pred_control_3([0.7051,1.21512,16.311,2.506],[0,0,0.6,0])
t=np.arange(0,0.2*50,0.2)
labels=['Cart Location','Cart Velocity','Pole Angle','Pole Velocity']
for i in range(4):
    plt.plot(t,aa[:,i],label ='[0,0,0.1,0')
    plt.plot(t,bb[:,i],label='[0,0,0.3,0')
    plt.xlabel('time(s')
    plt.ylabel(labels[i])
    plt.legend()
    plt.show()

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12, 6))
axs[0,0].plot(t,aa[:,0],color='black', label = '[0, 0, 0.1, 0]')
axs[0,0].plot(t,bb[:,0],color='royalblue',label = '[0,0,0.4,0]')
axs[0,0].plot(t,cc[:,0],color='red', label='[0,0,-0.2,0]')
axs[0,0].set_title('Time evolution of Cart Location')
axs[0,0].set(xlabel='time (s)',ylabel='Cart Location',)
axs[0,0].legend(loc='lower right')

axs[0,1].plot(t,aa[:,1],color='black',label = '[0, 0, 0.1, 0]')
axs[0,1].plot(t,bb[:,1],color='royalblue',label = '[0,0,0.4,0]')
axs[0,1].plot(t,cc[:,1],color='red',label='[0,0,-0.2,0]')
axs[0,1].set_title('Time evolution of Cart Velocity')
axs[0,1].set(xlabel='time (s)',ylabel='Cart Velocity')
axs[0,1].legend(loc='lower right')

axs[1,0].plot(t,aa[:,2],color='black',label = '[0, 0, 0.1, 0]')
axs[1,0].plot(t,bb[:,2],color='royalblue',label = '[0,0,0.4,0]')
axs[1,0].plot(t,cc[:,2],color='red',label='[0,0,-0.2,0]')
axs[1,0].set_title('Time evolution of Pole Angle')
axs[1,0].set(xlabel='time (s)',ylabel='Pole angle')
axs[1,0].legend(loc='lower right')

axs[1,1].plot(t,aa[:,3],color = 'black',label = '[0, 0, 0.1, 0]')
axs[1,1].plot(t,bb[:,3],color='royalblue',label = '[0,0,0.4,0]')
axs[1,1].plot(t,cc[:,3],color='red',label='[0,0,-0.2,0]')
axs[1,1].set_title('Time evolution of Pole Angular Velocity')
axs[1,1].set(xlabel='time (s)',ylabel='Pole Angular Velocity')
axs[1,1].set_ylim([-5,10])
axs[1,1].legend(loc='lower right')
fig.tight_layout()

# Task 2.4

In [ ]:
Kernel Function 
def kernel_func(x0,x1,sig):
    delta_x = x0-x1
    delta_x_2 = np.zeros([4,1])
    for i in range(4):
        delta_x_2[i]=(delta_x[i])**2
    delta_x_2[2]=np.sin(0.5*(delta_x[2]))**2
    expo = (-0.5)*float(np.dot(sig, delta_x_2))
    kernel = np.e**expo
    return kernel
#range of input for creating data
def r_range(i, minimum,maximum):
    d = i*(maximum-minimum)
    output = np.full(np.shape(i),minimum)+d
    return output
#generate datapoints using sobol_seq
def generate_x(n):
    x_data = sobol_seq.i4_sobol_generate(4,n)
    d_0=r_range(x_data[:,0],-50,50)
    d_1=r_range(x_data[:,1],-10,10)
    d_2=r_range(x_data[:,2],-(np.pi),np.pi)
    d_3=r_range(x_data[:,3],-15,15)
    output = np.column_stack((d_0,d_1,d_2,d_3))
    return output
def K_matrix(a,b,sig_invert):
    A = np.shape(a)[0]
    B = np.shape(b)[0]
    K = np.zeros(shape=(A,B))
    for i in range(A):
        for j in range(B):
            K[i][j] =kernel_func(a[i],b[j],sig_invert)
    return K

x= CartPole()
N =500
M = 200
sigma =[2.63690e-16,1.6274e-06, 2.6313, 1.3831e-02, 2.464e-04]
X = generate_x(N)
X_M = generate_x(M)
lamda = 1e-4
# generate a Nx4 matrix
# generate corresponding y
Y = np.zeros(shape=(N,4))
for i in range(N):
    x.setState(X[i,:4])
    x.performAction(X[i,-1])
    Y[i] = x.getState() - X[i,:4]

KNN = K_matrix(X,X, sigma)
KMN = K_matrix(X_M, X, sigma)
KNM = K_matrix(X, X_M, sigma)
KMM = K_matrix(X_M, X_M, sigma)
#alpha = (KMN@KNM + lamda*KMM)^-1(KMN@Y)
#alpha = (c)^-1*(d)
c = KMN @ KNM +lamda*KMM
d = np.zeros([4,M])
matrix_alphas = np.zeros([4,M])
for i in range(4):
    d[i] = KMN @ Y_change_State[:,i]
    matrix_alphas[i]=np.linalg.lstsq(c,d[i],rcond=-1)[0]
def model_pred_control(p):
    CartpoleState.setState([0, 0, 0.1, 0])
    pred = np.zeros(shape=(15,5))
    l = 0

    for i in range(15):
        l += CartpoleState.loss()
        f = np.dot(p, CartpoleState.getState())
        pred[i, :4] = CartpoleState.getState()
        pred[i, -1] = f
        K1M = np.array([kernel_func(pred[i], s,sigma) for s in X_M])
        c = K1M @ matrix_alphas.transpose()
        new_pred = c + pred[i,:4]
        CartpoleState.setState(new_pred)
        CartpoleState.remap_angle()
    return l


### Minimums

In [ ]:
n_s = 200
l = np.zeros([n_s,1])
p_cl=np.zeros([n_s,4])
for i in range(n_s):
    p1 = [np.random.uniform(0,100),1,1,1]
    p_cl[i] = p1
    l[i]=model_pred_control(p1)
l_cv = np.zeros([n_s,1])
p_cv=np.zeros([n_s,4])
for i in range(n_s):
    p2 = [1,np.random.uniform(0,100),1,1]
    p_cv[i] = p2
    l_cv[i]=model_pred_control(p2)
plt.scatter(p_cv[:,1],l_cv)
plt.ylim(10,14)
plt.xlabel('p2')
plt.ylabel('Loss')
plt.title('Varying p2')
plt.show()


### Time Evolution + Optimizing

In [ ]:
res = minimize(model_pred_control, p_1, method='nelder-mead',
               options={'maxiter':3000,'xatol': 1e-5, 'disp': True})
print(res.x)
def plot_evolution_policy(n, p):
    CartpoleState = CartPole()
    #Different intial states
    init_state_array = np.array([[0, 0, 0.2, 0],[0, 0, 0.12, 0], [0, 0, 0.1, 0], [0, 0, 0.15, 0]])

    x0 = np.zeros([4,n])
    x1 = np.zeros([4,n])
    x2 = np.zeros([4,n])
    x3 = np.zeros([4,n])
    x4 = np.zeros([4,n])

    index = 0
    for init in init_state_array:
        CartpoleState.setState(init)
        l = 0
        for i in range(n):
            cur_state = CartpoleState.getState()
            x0[ind][i], x1[ind][i], x2[ind][i], x3[ind][i] = cur_state[0], cur_state[1], cur_state[2], cur_state[3]
            l += CartpoleState.loss()
            f = np.dot(p, CartpoleState.getState())
            x4[ind][i] = f
            K1M_pred = np.array([kernel_gauss(np.array([x0[ind][i], x1[ind][i], x2[ind][i], x3[ind][i],x4[ind][i]]), sset, one_over_sigma) for sset in Xs])
            c = K1M_pred @ matrix_alphas.transpose()
            new_pred = c + cur_state
            CartpoleState.setState(new_pred)
            CartpoleState.remap_angle()
        index += 1
    X = np.array([x0, x1, x2, x3, x4])


    fig1, axs1 = plt.subplots(4,figsize=(10,5))
    fig1.suptitle('Time evolution of variables')

    time = np.arange(0, 0.2*n, 0.2)
    states = ["location", "velocity", "pole angle", "pole velocity"]

    index = 0
    for i in range(4):
        axs1[i].plot(time, X[index][0], color='royalblue',label = '[0, 0.2, 0, 0]')
        axs1[i].plot(time, X[index][2], color='red' ,label = '[0, 0, 0.1, 0]')
        axs1[i].plot(time, X[index][3],color = 'black', label = '[0, 0, 0.15, 0')
        axs1[i].set(xlabel="Time")
        axs1[i].set(ylabel=states[index])
        index += 1
    axs1[0].legend(bbox_to_anchor=(1.1, 1.3), loc='upper right')
    plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=180073fb-9c10-447b-b721-e7b1d25fe36a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>